<h1 style="text-align: center; background: #013D4F; color: white; padding: 15px 0">
    K-PROTOTYPES++
</h1>

<h2 style="text-align: center; background: #0A5F77; color: white; padding: 15px 0; margin: 0"> Índice </h2>
<div style="background: #F8FDFF; color: black; padding: 15px 0">
    <ul>
        <li><a href='#lib' style="color: black; text-decoration: none;">Importando librerías</a></li>
        <li><a href='#cd' style="color: black; text-decoration: none;">Conjunto de datos</a></li>
        <li><a href='#p' style="color: black; text-decoration: none;">Preprocesamiento</a></li>
        <li><a href='#c' style="color: black; text-decoration: none;">Clases</a></li>
        <ul>
            <li><a href='#c_est' style="color: black; text-decoration: none;">Estadística</a></li>
            <li><a href='#c_kplus' style="color: black; text-decoration: none;">K-PrototypesPlus</a></li>
            <li><a href='#c_r' style="color: black; text-decoration: none;">Reporte</a></li>
        </ul>
        <li><a href='#nda' style="color: black; text-decoration: none;">N° de agrupaciones</a></li>
        <li><a href='#impl' style="color: black; text-decoration: none;">Implementación de algoritmo</a></li>
        <ul>
            <li><a href='#kproto' style="color: black; text-decoration: none;">K-Prototypes</a></li>
            <ul>
                <li><a href='#m1' style="color: black; text-decoration: none;">Métrica de medición</a></li>
                <li><a href='#r1' style="color: black; text-decoration: none;">Reporte</a></li>
                <li><a href='#p1' style="color: black; text-decoration: none;">Pureza (si aplica)</a></li>
            </ul>
            <li><a href='#kproto_plus' style="color: black; text-decoration: none;">K-Prototypes++</a></li>
            <ul>
                <li><a href='#m2' style="color: black; text-decoration: none;">Métrica de medición</a></li>
                <li><a href='#r2' style="color: black; text-decoration: none;">Reporte</a></li>
                <li><a href='#p2' style="color: black; text-decoration: none;">Pureza (si aplica)</a></li>
            </ul>
        </ul>
    </ul>
</div>

<a id="lib">
<h2 style="text-align: center; background: #0A5F77; color: white; padding: 15px 0; margin: 0"> 
    Importando librerías 
</h2>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import LabelEncoder

<a id="cd">
<h2 style="text-align: center; background: #0A5F77; color: white; padding: 15px 0; margin: 0"> 
    Conjunto de datos
</h2>

In [ ]:
archivo = 'ad_campaign_performance.csv'
archivo = 'covid_variants_dataset.csv'
archivo = 'USA_cars_datasets.csv'
archivo = 'data_science_job.csv'
archivo = 'Energy_consumption_dataset.csv'

# Probar la pureza de los datos
archivo = 'condicion.csv'
archivo = 'ad_campaign_performance_pureza.csv'
archivo = 'tratamiento.csv'
archivo = 'salarios.csv'
archivo = 'pacientes.csv'
archivo = 'liver_cancer_prediction.csv'


# archivo = 'hypertension_dataset.csv'
# archivo = 'depresion_estudiantes.csv'

df = pd.read_csv(archivo, sep=',')

match archivo:
    case 'ad_campaign_performance.csv':
        df = df.iloc[:, 1:-1]
    case 'USA_cars_datasets.csv':
        df.loc[df['model'] == '1500', 'model'] = '_1500'
    case 'depresion_estudiantes.csv':
        df = df.iloc[:, 1:]
    case 'ad_campaign_performance_pureza.csv':
        df_con_etiqueta = df
        etiquetas_df = df.iloc[:, -1:]
        df = df.iloc[:, 1:]
    case _:
        df_con_etiqueta = df
        etiquetas_df = df.iloc[:, -1:]
        df = df.iloc[:, :-1]

df.head()

### Observamos cuantas filas y columnas tiene

In [ ]:
df.shape

### Analizamos de manera aleatoria nuestro conjunto de datos

In [ ]:
df.sample(7)

<a id="p">
<h2 style="text-align: center; background: #0A5F77; color: white; padding: 15px 0; margin: 0"> 
    Preprocesamiento
</h2>

### Construimos nuestro diccionario

In [ ]:
datadict = pd.DataFrame(df.dtypes)
datadict.columns = ['Tipo']
print(datadict)

### Manejo de datos perdidos

#### Buscando datos perdidos y duplicados

In [ ]:
numericos_df = df.select_dtypes(include=['number'])
categoricos_df = df.select_dtypes(include=['object', 'string'])
print(f'\tNuméricos nulos: \n{numericos_df.isnull().sum()}\n')
print(f'\tCategoricos nulos: \n{categoricos_df.isnull().sum()}\n')

#### Agregamos la columna de datos perdidos y duplicados a nuestro diccionario

In [ ]:
perdidos = df.isnull().sum()
datadict['Perdidos'] = perdidos
print(datadict)

In [ ]:
duplicados = df.duplicated().sum()
datadict['Duplicados'] = duplicados
datadict

#### Numéricos

In [ ]:
print(f'Numéricos nulos: \n{numericos_df.isnull().sum()}')

#### Aplicando algoritmo de imputación de datos nulos (si existieran)

In [ ]:
from sklearn.impute import SimpleImputer

# Imputación de datos numéricos
num_imputer = SimpleImputer(strategy='mean')
df[numericos_df.columns] = num_imputer.fit_transform(numericos_df)

### Categóricos

In [ ]:
print(f'Categoricos nulos: \n{categoricos_df.isnull().sum()}\n')

#### Aplicando algoritmo de imputación de datos nulos (si existieran)

In [ ]:
from sklearn.impute import SimpleImputer

# Imputación de datos categóricos
cat_imputer = SimpleImputer(strategy='most_frequent')
df[categoricos_df.columns] = cat_imputer.fit_transform(categoricos_df)

### Codificación de datos categóricos

<p sytle="text-align: justify;">
No es necesario codificar las variables categoricas, debido a que el algoritmo a implementar es aplicado con valores mixtos, sin embargo, para fines de estudio y prueba, veamos que pasa si codifican las varibales categoricas.
</p>

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# # Codificando las variables categoricas
# lb = LabelEncoder()
# for columna in categoricos_df.columns:
#     df[columna] = lb.fit_transform(df[columna])

<a id="c">
<h2 style="text-align: center; background: #0A5F77; color: white; padding: 15px 0; margin: 0"> 
    Clases
</h2>

<a id="c_est">
<h3 style="text-align: center; background: #008DA2; color: white; padding: 15px 0; margin: 0"> 
    Estadística
</h3>

In [ ]:
class Estadistica:
    @classmethod
    def distancia_euclidiana(cls, arrays, centro):
        distancias_list = []
        distancia = 0
        for arr in arrays:
            for i in range(len(arr)):
                distancia += (arr[i] - centro[i])**2
            distancias_list.append(round(np.sqrt(distancia), 2))
            distancia = 0
        return np.array(distancias_list)

    @classmethod
    def similitud(cls, arrays, centro):
        similitud_list = []
        similitud = 0
        for arr in arrays:
            for i in range(len(arr)):
                if arr[i] != centro[i]:
                    similitud += 1    
            similitud_list.append(similitud)
            similitud = 0
        return np.array(similitud_list)

    @classmethod
    def minimos(cls, *arrays):
        minimos = []
        for i in range(len(arrays[0])):
            valores = [arr[i] for arr in arrays]
            min_valor = min(valores)
            min_index = valores.index(min_valor)
            min_columna = min_index + 1
            minimos.append([min_valor, min_columna, i])
        return minimos

    @classmethod
    def promedios(cls, dataframe):
        columnas = dataframe.columns
        nuevo_centroide = []
        for columna in columnas:
            if dataframe[columna].dtype == 'object' or dataframe[columna].dtype == 'string':
                if len(dataframe[columna].mode()) > 1:
                    # Aquí se puede escoger la moda dependiendo de cada uno
                    nuevo_centroide.append(dataframe[columna].mode()[0])
                else:
                    nuevo_centroide.append(dataframe[columna].mode()[0])
                    
            else:
                nuevo_centroide.append(round(dataframe[columna].mean(), 2))
        return np.array(nuevo_centroide)

    @classmethod
    def normalizar_numericos(cls, df):
        # Separando datos categóricos y numéricos
        numericos_df = df.select_dtypes(['number'])
        categoricos_df = df.select_dtypes(['object','string'])
        # Normalizando los datos numéricos
        numericos_df = round((numericos_df - numericos_df.min()) / (numericos_df.max() - numericos_df.min()), 2)
        return numericos_df, categoricos_df

    @classmethod
    def is_numeric(cls, value):
        try:
            float(value)
            return True
        except ValueError:
            return False

<a id="c_kplus">
<h3 style="text-align: center; background: #008DA2; color: white; padding: 15px 0; margin: 0"> 
    KPrototypesPlus
</h3>

In [ ]:
class KPrototypesPlus:
    def __centroides_probabilidad(self, cent, df, peso, primera_pos):
        # Separando datos categóricos y numéricos (numéricos ya normalizados)
        numericos_df, categoricos_df = Estadistica.normalizar_numericos(df)
    
        # Generando los centroides (obteniendo el mejor centride de acuerdo a probabilidades)
        indice_max = -1
        indices = [primera_pos]
        distancias_finales = []
        for i in range(cent):
            # Localizando centroides
            centroide_numerico = numericos_df.iloc[primera_pos] if i == 0 else numericos_df.iloc[indice_max]
            centroide_categorico = categoricos_df.iloc[primera_pos] if i == 0 else categoricos_df.iloc[indice_max]
            
            # Calculando las distancias euclidianas y similitud de coeficientes
            d_e = Estadistica.distancia_euclidiana(numericos_df.to_numpy(), centroide_numerico.to_numpy())
            s = Estadistica.similitud(categoricos_df.to_numpy(), centroide_categorico.to_numpy())
            
            # Calulando las distancias finales
            dist_final = d_e + s * 1
            distancias_finales.append(dist_final)
        
            if not (len(distancias_finales) > 1):
                # Calculando el indice del nuevo centroide
                probabiliad = dist_final / sum(dist_final)
                max_cent = max(probabiliad)
                indice_max = np.where(probabiliad == max_cent)[0][0]
                indices.append(indice_max)
            else:
                # Obteniendo los minimos de las distancias
                m = Estadistica.minimos(*distancias_finales)
                m = np.array(np.array(m)[:, 0], dtype='float64')
                
                # Calculando el indice del nuevo centroide
                probabiliad = m / sum(m)
                max_cent = max(probabiliad)
                indice_max = np.where(probabiliad == max_cent)[0][0]
                indices.append(indice_max)
        return indices

    
    def _k_prototypes(self, k, df, peso, *args, inicio=1, fin=50, pos_fin=None):
        # Separando datos categóricos y numéricos (numéricos ya normalizados)
        numericos_df, categoricos_df = Estadistica.normalizar_numericos(df)
    
        # Generando los centroides
        centroides_numericos = []
        centroides_categoricos = []
        distancias_e = []
        similitud_coef = []
    
        if pos_fin is None:
            for i in range(k):
                # Numéricos
                centroides_numericos.append(numericos_df.iloc[args[i]])
                # Categóricos
                centroides_categoricos.append(categoricos_df.iloc[args[i]])
        
            # Calculando las distancias euclidianas y similitud de coeficientes
            for i in range(k):
                distancias_e.append(Estadistica.distancia_euclidiana(numericos_df.to_numpy(), centroides_numericos[i].to_numpy()))
                similitud_coef.append(Estadistica.similitud(categoricos_df.to_numpy(), centroides_categoricos[i].to_numpy()))
        else:
            fila_n = []
            fila_c = []
            for row in pos_fin:
                for dato in row:
                    if Estadistica.is_numeric(dato):    
                        fila_n.append(dato)
                    else:
                        fila_c.append(dato)
                centroides_numericos.append(fila_n) # Numéricos
                centroides_categoricos.append(fila_c) # Categóricos
                fila_n = []
                fila_c = []

    
            centroides_numericos = np.array(centroides_numericos, dtype='float64')
            centroides_categoricos = np.array(centroides_categoricos, dtype='object')
            # Calculando las distancias euclidianas y similitud de coeficientes
            for i in range(k):
                distancias_e.append(Estadistica.distancia_euclidiana(numericos_df.to_numpy(), centroides_numericos[i]))
                similitud_coef.append(Estadistica.similitud(categoricos_df.to_numpy(), centroides_categoricos[i]))
           
    
        # Calulando las distancias finales
        distancias_cent = []
        for i in range(k):
            dist_final = distancias_e[i] + similitud_coef[i] * peso
            distancias_cent.append(dist_final)
    
        # Obteniendo los minimos y sus etiquetas. Después conviertiendolo a un dataframe
        m = Estadistica.minimos(*distancias_cent)
        salida = pd.DataFrame(m, columns=['Minimo', 'Etiqueta', 'Indice'])
    
        # Calculando los promedios de cada centroide con su etiqueta
        etiquetas = [i+1 for i in range(k)]
        nuevos = []
        titulo = f' Iteración {inicio} '
        print(titulo.center(70, '-'))
        for i in range(k):
            indice = salida[salida['Etiqueta'] == etiquetas[i]]['Indice']
            nuevo_centriode = pd.concat([categoricos_df.iloc[indice], numericos_df.iloc[indice]], axis=1)
            # print('\n', nuevo_centriode)
            prom = Estadistica.promedios(nuevo_centriode)
            nuevos.append(prom)
            print(f'\nNuevo C{etiquetas[i]} = {prom}')
        print()
        # Validando recursividad
        if np.array_equal(nuevos, pos_fin) or inicio == fin:
            return nuevos, etiquetas, salida, pd.concat([categoricos_df, numericos_df], axis=1)
        else:
            return self._k_prototypes(k, df, peso, *args, inicio=inicio+1, pos_fin=nuevos)


    
    def k_prototypes_plus(self, df, k, peso, semilla):
        try:
            # Establecer la semilla
            np.random.seed(semilla)
            
            # Generar un número aleatorio entero dentro del rango de los datos
            numero_entero = np.random.randint(0, len(df))
            # print(numero_entero)
            # Calculando el k-prototypes++
            centroides = self.__centroides_probabilidad(cent=k-1, df=df, peso=peso, primera_pos=numero_entero) #  Obteniendo los centroides
            print(' K-PROTOTYPES++ '.center(100, '_'), '\n')
            print('\t\t\t\tCentroides en las posiciones:', centroides, '\n')
            return self._k_prototypes(k, df, peso, *centroides)
        except Exception as e:
            print(f'Ocurrio un error: {e}')
            return -1


    def silhouette(self, centroide_final):
        # Recupeperando los datos noramalizados
        df_etiquetas = centroide_final[-1]
        # salida (concatenando sus etiquetas)
        df_etiquetas['Etiqueta'] = centroide_final[2]['Etiqueta'] 
        
        # Obtienendo el df normalizado y sus etiquetas
        etiquetas = df_etiquetas.iloc[:, -1]
        datos = df_etiquetas.iloc[:, :-1]
        
        # Codificando las variables categoricas
        lb = LabelEncoder()
        columnas_categoricas = df.select_dtypes(include=['object', 'string']).columns
        for columna in columnas_categoricas:
            datos[columna] = lb.fit_transform(datos[columna]) # codificando datos
        
        datos = Estadistica.normalizar_numericos(datos)[0] # normalizando los datos ([0] obtiene los numericos)
        
        # Calcular el Silhouette Score para K-Prototypes
        sil_score = round(silhouette_score(datos, etiquetas, metric='euclidean'), 2)
        print(f'Silhouette: {sil_score}')
        
        return pd.DataFrame({'Silhouette': [sil_score]}) # Creando su df para colocarlo en el reporte



    def tipo_etiqueta(self, etiquetas_df):
        # Obteniendo la columna donde estan las etiquetas
        columna = etiquetas_df.columns[0]
        unicos = etiquetas_df[columna].unique()
        etiquetas = {}
        # Iterando cada valor para asignarle el centroide (numérico)
        for i, valor in enumerate(unicos):
            etiquetas[valor] = i + 1
        # Creando el df ya con el tipo de etiqueta correspondiente
        etiquetas_original_df = pd.DataFrame(etiquetas_df[columna].map(etiquetas))
        return etiquetas_original_df
    
    def pureza(self, centroide_final, etiquetas_original_df):
        # Obteniendo las etiquetas de nuestro resultado
        etiq_resultado_df = centroide_final[3].iloc[:, -1:]
        # Comparando la certeza del algoritmo
        comparacion = etiq_resultado_df[etiq_resultado_df.columns[0]] == etiquetas_original_df[etiquetas_original_df.columns[0]]
        # Promediando los resulados
        promedio = round(comparacion.mean()*100, 2)
        print(f'Pureza: {promedio}%')
        return pd.DataFrame({'Pureza': [promedio]})

<a id="c_r">
<h3 style="text-align: center; background: #008DA2; color: white; padding: 15px 0; margin: 0"> 
    Reporte
</h3>

In [ ]:
class Reporte:
    def __init__(self, nombre):
        self.archivo_excel = f'Reportes\\{nombre}.xlsx'

    def exportar_archivo(self, centroide_final):
        self.df_etiquetas = centroide_final[-1] # Recupeperando los datos noramalizados
        self.df_etiquetas['Etiqueta'] = centroide_final[2]['Etiqueta'] # salida (concatenando sus etiquetas)
        
        self.df_etiquetas.to_excel(self.archivo_excel, index=False) # Exportando a un archivo 

    def centroides_finales(self, centroide_final):
        # Uniendo las etiquetas de los centroies finales y colocandolo en un DataFrame
        centroides, etiquetas = centroide_final[0], centroide_final[1]
        promedio_centroides = []
        for i, _ in enumerate(centroides):
            centroide = centroides[i].tolist()
            centroide.append(etiquetas[i])
            promedio_centroides.append(centroide)
        promedio_centroides = pd.DataFrame(promedio_centroides, columns=self.df_etiquetas.columns)
        return promedio_centroides

    def anexando_informacion(self, df):
        # Si el archivo ya existe, anexamos los nuevos datos
        with pd.ExcelWriter(self.archivo_excel, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            # promedio_centroides.to_excel(writer, index=False, sheet_name='Sheet1', startrow=writer.sheets['Sheet1'].max_row, header=False) # fila
            df.to_excel(writer, index=False, sheet_name='Sheet1', startcol=writer.sheets['Sheet1'].max_column + 1, header=True) # columna

<a id="nda">
<h2 style="text-align: center; background: #0A5F77; color: white; padding: 15px 0; margin: 0"> 
    N° de agrupaciones
</h2>

In [ ]:
from kmodes.kprototypes import KPrototypes

df_mc = df.iloc[:, :]

# Codificando las variables categóricas
lb = LabelEncoder()
columnas_categoricas = df_mc.select_dtypes(include=['object', 'string']).columns


# Codificando las columnas categóricas
for columna in columnas_categoricas:
    # Ajustamos y transformamos la columna con LabelEncoder
    df_mc[columna] = lb.fit_transform(df[columna])  # Codificar la columna
    df_mc[columna] = df_mc[columna].astype('object')  # Asegurarse de que las columnas sean del tipo 'object'


# Indices de las columnas categóricas
indices_numericos_categoricas = [df_mc.columns.get_loc(col) for col in columnas_categoricas]

# Lista para almacenar la inercia
inertia = []

# Numpero máximo de clusters a probar
n = 7

for k in range(1, n):
    kproto = KPrototypes(n_clusters=k, init='Cao', random_state=42)
    kproto.fit(df_mc, categorical=indices_numericos_categoricas)
    inertia.append(kproto.cost_)

# Graficar la inercia vs. el número de clusters
plt.figure(figsize=(8, 6))
plt.plot(range(1, n), inertia, marker='o')
plt.title('Método del Codo para K-Prototypes')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Inercia')
plt.show()


<a id="impl">
<h2 style="text-align: center; background: #0A5F77; color: white; padding: 15px 0; margin: 0"> 
    Implementación de algoritmo
</h2>

### Creando una instancia de la clase y definiendo k y peso

In [ ]:
k_prototypes_plus = KPrototypesPlus()
k = 2
peso = 1

<a id="kproto">
<h3 style="text-align: center; background: #008DA2; color: white; padding: 15px 0; margin: 0"> 
    K-Prototypes
</h3>

In [ ]:
from random import randint
centroides = []
while len(centroides) != k:
    c = randint(df.index[0], df.index[-1])
    if c not in centroides:
        centroides.append(c)
    else:
        continue

# Calculando el k-prototypes
print(' K-PROTOTYPES '.center(100, '_'), '\n')
print('\t\t\t\tCentroides en las posiciones:', centroides, '\n')
centroide_final_k = k_prototypes_plus._k_prototypes(k, df, peso, *centroides)    

In [ ]:
# centroide_final

<a id="m1">
<h4 style="text-align: center; background: #D6F3FF; color: black; padding: 15px 0; margin: 0"> 
    Métrica de medición
</h4>

In [ ]:
sil_df_k = k_prototypes_plus.silhouette(centroide_final_k)

<a id="r1">
<h4 style="text-align: center; background: #D6F3FF; color: black; padding: 15px 0; margin: 0"> 
    Reporte
</h4>

In [ ]:
reporte = Reporte(f'{archivo[:-4]}_KPrototypes')
reporte.exportar_archivo(centroide_final_k)
cent_finales_etiquetas_df_k = reporte.centroides_finales(centroide_final_k)
reporte.anexando_informacion(cent_finales_etiquetas_df_k)
reporte.anexando_informacion(sil_df_k)

<a id="p1">
<h4 style="text-align: center; background: #D6F3FF; color: black; padding: 15px 0; margin: 0"> 
    Pureza (si aplica)
</h4>

In [ ]:
etiquetas_original_df = k_prototypes_plus.tipo_etiqueta(etiquetas_df)
pureza = k_prototypes_plus.pureza(centroide_final_k, etiquetas_original_df)
reporte.anexando_informacion(pureza)

<a id="kproto_plus">
<h3 style="text-align: center; background: #008DA2; color: white; padding: 15px 0; margin: 0"> 
    K-Prototypes++
</h3>

In [ ]:
centroide_final_kplus = k_prototypes_plus.k_prototypes_plus(df=df, k=k, peso=peso, semilla=13)

In [ ]:
# centroide_final

<a id="m2">
<h4 style="text-align: center; background: #D6F3FF; color: black; padding: 15px 0; margin: 0"> 
    Métrica de medición
</h4>

In [ ]:
sil_df_kplus = k_prototypes_plus.silhouette(centroide_final_kplus)

<a id="r2">
<h4 style="text-align: center; background: #D6F3FF; color: black; padding: 15px 0; margin: 0"> 
    Reporte
</h4>

In [ ]:
reporte = Reporte(f'{archivo[:-4]}_KPrototypes_Plus')
reporte.exportar_archivo(centroide_final_kplus)
cent_finales_etiquetas_df_kplus = reporte.centroides_finales(centroide_final_kplus)
reporte.anexando_informacion(cent_finales_etiquetas_df_kplus)
reporte.anexando_informacion(sil_df_kplus)

<a id="p2">
<h4 style="text-align: center; background: #D6F3FF; color: black; padding: 15px 0; margin: 0"> 
    Pureza (si aplica)
</h4>

In [ ]:
etiquetas_original_df = k_prototypes_plus.tipo_etiqueta(etiquetas_df)
pureza = k_prototypes_plus.pureza(centroide_final_kplus, etiquetas_original_df)
reporte.anexando_informacion(pureza)